In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sn
from pyspark.sql.functions import isnull, when, count
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark.sql.types import *

In [0]:
# Inspect the Mount's Final Project folder 
display(dbutils.fs.ls("/mnt/mids-w261/"))

path,name,size
dbfs:/mnt/mids-w261/HW5/,HW5/,0
dbfs:/mnt/mids-w261/datasets_final_project/,datasets_final_project/,0


In [0]:
display(dbutils.fs.ls("/mnt/mids-w261/datasets_final_project/parquet_airlines_data/"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/,2015.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2016.parquet/,2016.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2017.parquet/,2017.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2018.parquet/,2018.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2019.parquet/,2019.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/airlines_size_test.parquet/,airlines_size_test.parquet/,0


In [0]:
display(dbutils.fs.ls("/mnt/mids-w261/datasets_final_project/weather_data/"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather-miss.parquet/,weather-miss.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2015a.parquet/,weather2015a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2016a.parquet/,weather2016a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2017a.parquet/,weather2017a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2018a.parquet/,weather2018a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2019a.parquet/,weather2019a.parquet/,0


In [0]:
display(dbutils.fs.ls("/mnt/team11/"))

path,name,size
dbfs:/mnt/team11/5y_normalized/,5y_normalized/,0
dbfs:/mnt/team11/5y_normalized_v2/,5y_normalized_v2/,0
dbfs:/mnt/team11/airlines_2015transformed/,airlines_2015transformed/,0
dbfs:/mnt/team11/airlines_2016transformed/,airlines_2016transformed/,0
dbfs:/mnt/team11/airlines_2017transformed/,airlines_2017transformed/,0
dbfs:/mnt/team11/airlines_2018transformed/,airlines_2018transformed/,0
dbfs:/mnt/team11/airlines_2019transformed/,airlines_2019transformed/,0
dbfs:/mnt/team11/airlines_3m_transformed/,airlines_3m_transformed/,0
dbfs:/mnt/team11/airlines_5ytransformed/,airlines_5ytransformed/,0
dbfs:/mnt/team11/all_flight_weather_3m/,all_flight_weather_3m/,0


# Import Data

In [0]:
def import_data(timeframe):
    """
    timeframe: String. 
        If timeframe = 3m, then return airlines for flights departing from ORD and ATL only and weather data for the first quarter of 2015
        If timeframe = 6m, then return airlines for flights departing from ORD and ATL only and weather data for the first quarter of 2015
        If timeframe = 5y, then return airlines for all US domestic flights and weather data for 2015-2019
    return: airlines Spark Dataframe, weather Spark Dataframe
    """
    airline_directory = "/mnt/mids-w261/datasets_final_project/parquet_airlines_data"
    weather_directory = "/mnt/mids-w261/datasets_final_project/weather_data/*"
    if timeframe == "3m":
        df_airlines = spark.read.parquet(airline_directory + "_3m/")
        df_weather = spark.read.parquet(weather_directory).filter(f.col('DATE') < "2015-04-01T00:00:00.000")
    elif timeframe == "6m":
        df_airlines = spark.read.parquet(airline_directory + "_6m/")
        df_weather = spark.read.parquet(weather_directory).filter(f.col('DATE') < "2015-07-01T00:00:00.000")
    elif timeframe == "5y":
        df_airlines = spark.read.parquet(airline_directory + "/201*.parquet/")
        df_weather = spark.read.parquet(weather_directory).filter(f.col('DATE') < "2020-01-01T00:00:00.000")
    else:
        year = int(timeframe)
        df_airlines = spark.read.parquet(airline_directory + "/"+ timeframe + ".parquet/")
        df_weather = spark.read.parquet(weather_directory).filter((f.col('DATE') < str(year+1)+"-01-01T00:00:00.000") & \
                                                                  (f.col('DATE') >= str(year)+"-01-01T00:00:00.000"))
    return df_airlines, df_weather

In [0]:
# Define timeframe
timeframe = "5y"

In [0]:
# Load flights and weather
df_airlines, df_weather = import_data(timeframe)


In [0]:
display(df_airlines.limit(50))

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,DIV_ARR_DELAY,DIV_DISTANCE,DIV1_AIRPORT,DIV1_AIRPORT_ID,DIV1_AIRPORT_SEQ_ID,DIV1_WHEELS_ON,DIV1_TOTAL_GTIME,DIV1_LONGEST_GTIME,DIV1_WHEELS_OFF,DIV1_TAIL_NUM,DIV2_AIRPORT,DIV2_AIRPORT_ID,DIV2_AIRPORT_SEQ_ID,DIV2_WHEELS_ON,DIV2_TOTAL_GTIME,DIV2_LONGEST_GTIME,DIV2_WHEELS_OFF,DIV2_TAIL_NUM,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV3_AIRPORT_SEQ_ID,DIV3_WHEELS_ON,DIV3_TOTAL_GTIME,DIV3_LONGEST_GTIME,DIV3_WHEELS_OFF,DIV3_TAIL_NUM,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV4_AIRPORT_SEQ_ID,DIV4_WHEELS_ON,DIV4_TOTAL_GTIME,DIV4_LONGEST_GTIME,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
2019,2,6,1,6,2019-06-01,B6,20409,B6,N929JB,620,14679,1467903,33570,SAN,"San Diego, CA",CA,6,California,91,10721,1072102,30721,BOS,"Boston, MA",MA,25,Massachusetts,13,1449,1501,12.0,12.0,0.0,0,1400-1459,18.0,1519,2315,5.0,2324,2320,-4.0,0.0,0.0,-1,2300-2359,0.0,null,0.0,335.0,319.0,296.0,1.0,2588.0,11,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,1,6,2019-06-01,B6,20409,B6,N981JT,623,12478,1247805,31703,JFK,"New York, NY",NY,36,New York,22,12892,1289208,32575,LAX,"Los Angeles, CA",CA,6,California,91,1445,1439,-6.0,0.0,0.0,-1,1400-1459,14.0,1453,1736,6.0,1800,1742,-18.0,0.0,0.0,-2,1800-1859,0.0,null,0.0,375.0,363.0,343.0,1.0,2475.0,10,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,1,6,2019-06-01,B6,20409,B6,N969JT,624,12892,1289208,32575,LAX,"Los Angeles, CA",CA,6,California,91,12478,1247805,31703,JFK,"New York, NY",NY,36,New York,22,1450,1444,-6.0,0.0,0.0,-1,1400-1459,12.0,1456,2256,10.0,2322,2306,-16.0,0.0,0.0,-2,2300-2359,0.0,null,0.0,332.0,322.0,300.0,1.0,2475.0,10,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,1,6,2019-06-01,B6,20409,B6,N339JB,626,10821,1082106,30852,BWI,"Baltimore, MD",MD,24,Maryland,35,10721,1072102,30721,BOS,"Boston, MA",MA,25,Massachusetts,13,1105,1055,-10.0,0.0,0.0,-1,1100-1159,16.0,1111,1210,4.0,1228,1214,-14.0,0.0,0.0,-1,1200-1259,0.0,null,0.0,83.0,79.0,59.0,1.0,369.0,2,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,1,6,2019-06-01,B6,20409,B6,N997JL,629,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,10732,1073203,30732,BQN,"Aguadilla, PR",PR,72,Puerto 

In [0]:
df_airlines.printSchema()

root
-- YEAR: integer (nullable = true)
-- QUARTER: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: string (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
-- OP_CARRIER: string (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- OP_CARRIER_FL_NUM: integer (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- ORIGIN_STATE_FIPS: integer (nullable = true)
-- ORIGIN_STATE_NM: string (nullable = true)
-- ORIGIN_WAC: integer (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
-- DEST_CITY_MARKET_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- DEST_STATE_FIPS: integer (nullable = true)
-- DEST_STATE_NM: string (nullable = true)
-- DEST_WAC: integer (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: integer (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CANCELLED: double (nullable = true)
-- CANCELLATION_CODE: string (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)
-- FIRST_DEP_TIME: integer (nullable = true)
-- TOTAL_ADD_GTIME: double (nullable = true)
-- LONGEST_ADD_GTIME: double (nullable = true)
-- DIV_AIRPORT_LANDINGS: integer (nullable = true)
-- DIV_REACHED_DEST: double (nullable = true)
-- DIV_ACTUAL_ELAPSED_TIME: double (nullable = true)
-- DIV_ARR_DELAY: double (nullable = true)
-- DIV_DISTANCE: double (nullable = true)
-- DIV1_AIRPORT: string (nullable = true)
-- DIV1_AIRPORT_ID: integer (nullable = true)
-- DIV1_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV1_WHEELS_ON: integer (nullable = true)
-- DIV1_TOTAL_GTIME: double (nullable = true)
-- DIV1_LONGEST_GTIME: double (nullable = true)
-- DIV1_WHEELS_OFF: integer (nullable = true)
-- DIV1_TAIL_NUM: string (nullable = true)
-- DIV2_AIRPORT: string (nullable = true)
-- DIV2_AIRPORT_ID: integer (nullable = true)
-- DIV2_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV2_WHEELS_ON: integer (nullable = true)
-- DIV2_TOTAL_GTIME: double (nullable = true)
-- DIV2_LONGEST_GTIME: double (nullable = true)
-- DIV2_WHEELS_OFF: integer (nullable = true)
-- DIV2_TAIL_NUM: string (nullable = true)
-- DIV3_AIRPORT: string (nullable = true)
-- DIV3_AIRPORT_ID: integer (nullable = true)
-- DIV3_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV3_WHEELS_ON: integer (nullable = true)
-- DIV3_TOTAL_GTIME: double (nullable = true)
-- DIV3_LONGEST_GTIME: double (nullable = true)
-- DIV3_WHEELS_

In [0]:
display(df_airlines.describe())

summary,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,DIV_ARR_DELAY,DIV_DISTANCE,DIV1_AIRPORT,DIV1_AIRPORT_ID,DIV1_AIRPORT_SEQ_ID,DIV1_WHEELS_ON,DIV1_TOTAL_GTIME,DIV1_LONGEST_GTIME,DIV1_WHEELS_OFF,DIV1_TAIL_NUM,DIV2_AIRPORT,DIV2_AIRPORT_ID,DIV2_AIRPORT_SEQ_ID,DIV2_WHEELS_ON,DIV2_TOTAL_GTIME,DIV2_LONGEST_GTIME,DIV2_WHEELS_OFF,DIV2_TAIL_NUM,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV3_AIRPORT_SEQ_ID,DIV3_WHEELS_ON,DIV3_TOTAL_GTIME,DIV3_LONGEST_GTIME,DIV3_WHEELS_OFF,DIV3_TAIL_NUM,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV4_AIRPORT_SEQ_ID,DIV4_WHEELS_ON,DIV4_TOTAL_GTIME,DIV4_LONGEST_GTIME,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
count,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31674354,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31274521,31269545,31269545,31269545,31269545,31746841,31260424,31260429,31244917,31244917,31746841,31244919,31176201,31176201,31176201,31176201,31746841,31746841,489947,31746841,31746677,31178799,31178799,31746841,31746841,31746841,5799114,5799114,5799114,5799114,5799114,203262,203260,203260,31746838,78108,66118,66119,78096,81641,81641,81641,81640,81641,81641,66537,66539,687,691,691,687,687,687,277,277,7,7,7,7,7,7,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
mean,2017.1512498204152,2.5174877084620797,6.552106365480585,15.749554640727876,3.9346285509162944,null,null,19949.20515190787,null,8806.54128440367,2339.5700956514065,12668.724409461716,1266875.803290192,31729.315288031336,null,null,null,26.35374732245013,null,54.91906164774001,12668.666651116562,1266870.0274082704,31729.2951808339,null,null,null,26.354102948384693,null,54.919218135750896,1330.0884999550035,1334.2122192375064,9.855285614165476,12.909587811399238,0.18207943223989986,0.036036885090588944,null,16.830789563186986,1356.9563268309594,1464.4766360877195,7.5604571777227,1488.9034405659447,1468.8957719173475,4.615475952313754,12.966188215170925,0.18601095752494026,-0.20968071125792395,null,0.015432937091284137,null,0.0024603392822611864,143.2167191860742,138.22906985609035,113.8502422431345,1.0,823.2170183483768,3.7652922065537164,19.98459350859459,3.2259498606166392,15.44036813209742,0.08916793151505557,25.364284785572416,1330.4193110369868,37.360331595001476,36.78451244711207,0.003480000118436992,0.8465074000102423,381.64725188299707,234.13669293243998,42.599236836713786,null,12717.885204737815,1271791.6214034616,1509.759015188633,35.31460908122145,28.555541945836037,1540.3529164224417,8805.0,null,12468.658465991317,1246869.2836468886,1384.3318777292577,22.676855895196507,18.86754002911208,1367.2454873646209,null,null,13244.285714285714,1324431.4285714286,1475.2857142857142,15.857142857142858,13.428571428571429,853.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
stddev,1.4316532810214855,1.1053295681781916,3.3994302561415655,8.774238088354572,1.9917635387471728,null,null,383.1919615319282,null

In [0]:
display(df_weather.limit(50))

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AW1,GA1,GA2,GA3,GA4,GE1,GF1,KA1,KA2,MA1,MD1,MW1,MW2,OC1,OD1,OD2,REM,EQD,AW2,AX4,GD1,AW5,GN1,AJ1,AW3,MK1,KA4,GG3,AN1,RH1,AU5,HL1,OB1,AT8,AW7,AZ1,CH1,RH3,GK1,IB1,AX1,CT1,AK1,CN2,OE1,MW5,AO1,KA3,AA3,CR1,CF2,KB2,GM1,AT5,AY2,MW6,MG1,AH6,AU2,GD2,AW4,MF1,AA1,AH2,AH3,OE3,AT6,AL2,AL3,AX5,IB2,AI3,CV3,WA1,GH1,KF1,CU2,CT3,SA1,AU1,KD2,AI5,GO1,GD3,CG3,AI1,AL1,AW6,MW4,AX6,CV1,ME1,KC2,CN1,UA1,GD5,UG2,AT3,AT4,GJ1,MV1,GA5,CT2,CG2,ED1,AE1,CO1,KE1,KB1,AI4,MW3,KG2,AA2,AX2,AY1,RH2,OE2,CU3,MH1,AM1,AU4,GA6,KG1,AU3,AT7,KD1,GL1,IA1,GG2,OD3,UG1,CB1,AI6,CI1,CV2,AZ2,AD1,AH1,WD1,AA4,KC1,IA2,CF3,AI2,AT1,GD4,AX3,AH4,KB3,CU1,CN4,AT2,CG1,CF1,GG1,MV2,CW1,GG4,AB1,AH5,CN3
7650099999,2016-01-01T00:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-12,99999,V020,"190,1,N,0015,1","99999,9,9,N","007000,1,9,9","+0101,1","+0090,1","10255,1",,"99,9,+02250,1,99,9",,,,"9,AGL ,+99999,+99999","08,99,1,99,9,99,9,02250,1,99,9,99,9",,,"99999,9,10216,1","8,1,004,1,+999,9","61,1",,,"3,99,0026,1,999",,SYN09807650 04857 81903 10101 20090 30216 40255 58004 69901 761// 333 4/000 69907 90710 91105 555 69905=,,,,,,,"9999,1,9,999999,9,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"03,0000,2,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"06,0000,2,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T00:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-15,99999,V020,"190,1,N,0015,1","22000,1,9,N","009000,1,9,9","+0100,1","+0090,1","99999,9","61,1",,,,,,"00,99,1,99,9,99,9,99999,9,99,9,99,9",,,"10250,1,99999,9",,,,,,,MET057METAR LFML 010000Z AUTO 19003KT 9000 -RA NSC 10/09 Q1025=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T00:30:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-15,99999,V020,"250,1,N,0010,1","99999,9,9,N","008000,1,9,9","+9999,9","+9999,9","99999,9",,,,,,,,,,"10250,1,99999,9",,,,,,,MET056METAR LFML 010030Z AUTO 25002KT 8000 ///TCU 10/09 Q1025=,Q019 2ATOD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T01:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-12,99999,V020,"999,9,C,0000,1","99999,9,9,N","004900,1,9,9","+0099,1","+0094,1","10251,1",,"99,9,+02250,1,99,9",,,,"9,AGL ,+99999,+99999","08,99,1,99,9,99,9,02250,1,99,9,99,9",,,"99999,9,10212,1","8,1,006,1,+999,9","10,1",,,"3,99,0015,1,999",,SYN07607650 24849 80000 10099 20094 30212 40251 58006 710// 333 69925 90710 91103=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"01,0002,3,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T01:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-15,99999,V020,"999,9,C,0000,1","22000,1,9,N","007000,1,9,9","+0100,1","+0090,1","99999,9",,,,,,,"00,99,1,99,9,99,9,99999,9,99,9,99,9",,,"10250,1,99999,9",,,,,,,MET053METAR LFML 010100Z AUTO 00000KT 7000 NSC 10/09 Q1025=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T01:30:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-15,99999,V020,"999,9,C,0000,1","22000,1,9,N","009000,1,9,9","+0100,1","+0090,1","99999,9",,,,,,,"00,99,1,99,9,99,9,99999,9,99,9,99,9",,,"10250,1,99999,9",,,,,,,MET053METAR LFML 010130Z AUTO 00000KT 9000 NSC 10/09 Q1025=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T02:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-12,99999,V020,"999,9,C,0000,1","99999,9,9,N","011000,1,9,9","+0100,1","+0088,1","10254,1",,"99,9,+01750,1,99,9",,,,"9,AGL ,+99999,+99999","08,99,1,99,9,99,9,01750,1,99,9,99,9",,,"99

In [0]:
df_weather.printSchema()

root
-- STATION: string (nullable = true)
-- DATE: timestamp (nullable = true)
-- SOURCE: short (nullable = true)
-- LATITUDE: double (nullable = true)
-- LONGITUDE: double (nullable = true)
-- ELEVATION: double (nullable = true)
-- NAME: string (nullable = true)
-- REPORT_TYPE: string (nullable = true)
-- CALL_SIGN: string (nullable = true)
-- QUALITY_CONTROL: string (nullable = true)
-- WND: string (nullable = true)
-- CIG: string (nullable = true)
-- VIS: string (nullable = true)
-- TMP: string (nullable = true)
-- DEW: string (nullable = true)
-- SLP: string (nullable = true)
-- AW1: string (nullable = true)
-- GA1: string (nullable = true)
-- GA2: string (nullable = true)
-- GA3: string (nullable = true)
-- GA4: string (nullable = true)
-- GE1: string (nullable = true)
-- GF1: string (nullable = true)
-- KA1: string (nullable = true)
-- KA2: string (nullable = true)
-- MA1: string (nullable = true)
-- MD1: string (nullable = true)
-- MW1: string (nullable = true)
-- MW2: string (nullable = true)
-- OC1: string (nullable = true)
-- OD1: string (nullable = true)
-- OD2: string (nullable = true)
-- REM: string (nullable = true)
-- EQD: string (nullable = true)
-- AW2: string (nullable = true)
-- AX4: string (nullable = true)
-- GD1: string (nullable = true)
-- AW5: string (nullable = true)
-- GN1: string (nullable = true)
-- AJ1: string (nullable = true)
-- AW3: string (nullable = true)
-- MK1: string (nullable = true)
-- KA4: string (nullable = true)
-- GG3: string (nullable = true)
-- AN1: string (nullable = true)
-- RH1: string (nullable = true)
-- AU5: string (nullable = true)
-- HL1: string (nullable = true)
-- OB1: string (nullable = true)
-- AT8: string (nullable = true)
-- AW7: string (nullable = true)
-- AZ1: string (nullable = true)
-- CH1: string (nullable = true)
-- RH3: string (nullable = true)
-- GK1: string (nullable = true)
-- IB1: string (nullable = true)
-- AX1: string (nullable = true)
-- CT1: string (nullable = true)
-- AK1: string (nullable = true)
-- CN2: string (nullable = true)
-- OE1: string (nullable = true)
-- MW5: string (nullable = true)
-- AO1: string (nullable = true)
-- KA3: string (nullable = true)
-- AA3: string (nullable = true)
-- CR1: string (nullable = true)
-- CF2: string (nullable = true)
-- KB2: string (nullable = true)
-- GM1: string (nullable = true)
-- AT5: string (nullable = true)
-- AY2: string (nullable = true)
-- MW6: string (nullable = true)
-- MG1: string (nullable = true)
-- AH6: string (nullable = true)
-- AU2: string (nullable = true)
-- GD2: string (nullable = true)
-- AW4: string (nullable = true)
-- MF1: string (nullable = true)
-- AA1: string (nullable = true)
-- AH2: string (nullable = true)
-- AH3: string (nullable = true)
-- OE3: string (nullable = true)
-- AT6: string (nullable = true)
-- AL2: string (nullable = true)
-- AL3: string (nullable = true)
-- AX5: string (nullable = true)
-- IB2: string (nullable = true)
-- AI3: string (nullable = true)
-- CV3: string (nullable = true)
-- WA1: string (nullable = true)
-- GH1: string (nullable = true)
-- KF1: string (nullable = true)
-- CU2: string (nullable = true)
-- CT3: string (nullable = true)
-- SA1: string (nullable = true)
-- AU1: string (nullable = true)
-- KD2: string (nullable = true)
-- AI5: string (nullable = true)
-- GO1: string (nullable = true)
-- GD3: string (nullable = true)
-- CG3: string (nullable = true)
-- AI1: string (nullable = true)
-- AL1: string (nullable = true)
-- AW6: string (nullable = true)
-- MW4: string (nullable = true)
-- AX6: string (nullable = true)
-- CV1: string (nullable = true)
-- ME1: string (nullable = true)
-- KC2: string (nullable = true)
-- CN1: string (nullable = true)
-- UA1: string (nullable = true)
-- GD5: string (nullable = true)
-- UG2: string (nullable = true)
-- AT3: string (nullable = true)
-- AT4: string (nullable = true)
-- GJ1: string (nullable = true)
-- MV1: string (nullable = true)
-- GA5: string (nullable = true)
-- CT2: string (nullable = true)
-- CG2: string

In [0]:
display(df_weather.describe())

summary,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AW1,GA1,GA2,GA3,GA4,GE1,GF1,KA1,KA2,MA1,MD1,MW1,MW2,OC1,OD1,OD2,REM,EQD,AW2,AX4,GD1,AW5,GN1,AJ1,AW3,MK1,KA4,GG3,AN1,RH1,AU5,HL1,OB1,AT8,AW7,AZ1,CH1,RH3,GK1,IB1,AX1,CT1,AK1,CN2,OE1,MW5,AO1,KA3,AA3,CR1,CF2,KB2,GM1,AT5,AY2,MW6,MG1,AH6,AU2,GD2,AW4,MF1,AA1,AH2,AH3,OE3,AT6,AL2,AL3,AX5,IB2,AI3,CV3,WA1,GH1,KF1,CU2,CT3,SA1,AU1,KD2,AI5,GO1,GD3,CG3,AI1,AL1,AW6,MW4,AX6,CV1,ME1,KC2,CN1,UA1,GD5,UG2,AT3,AT4,GJ1,MV1,GA5,CT2,CG2,ED1,AE1,CO1,KE1,KB1,AI4,MW3,KG2,AA2,AX2,AY1,RH2,OE2,CU3,MH1,AM1,AU4,GA6,KG1,AU3,AT7,KD1,GL1,IA1,GG2,OD3,UG1,CB1,AI6,CI1,CV2,AZ2,AD1,AH1,WD1,AA4,KC1,IA2,CF3,AI2,AT1,GD4,AX3,AH4,KB3,CU1,CN4,AT2,CG1,CF1,GG1,MV2,CW1,GG4,AB1,AH5,CN3
count,626994336,548964297,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436,630904436
mean,5.900842664264083E10,4.945690792346738,36.822635077084044,-36.7002829955328,369.4329781232751,null,null,99999.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
stddev,3.273023977302337E10,1.3644544181414908,22.61739266556294,78.71810434001519,534.4094770398771,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null

In [0]:
# Load stations dataframe
df_stations = spark.read.parquet("/mnt/mids-w261/datasets_final_project/stations_data/*")


In [0]:
display(df_stations.limit(50))

usaf,wban,station_id,lat,lon,neighbor_id,neighbor_name,neighbor_state,neighbor_call,neighbor_lat,neighbor_lon,distance_to_neighbor
690020,93218,69002093218,36.0,-121.233,69002093218,JOLON HUNTER LIGGETT MIL RES,CA,KHGT,36.0,-121.233,0.0
690020,93218,69002093218,36.0,-121.233,69007093217,FRITZSCHE AAF,CA,KOAR,36.683,-121.767,55.73024537916726
690020,93218,69002093218,36.0,-121.233,69014093101,EL TORO MCAS,CA,KNZJ,33.667,-117.733,255.49106220353934
690020,93218,69002093218,36.0,-121.233,70027127506,BARROW POINT BARROW,AK,KPBA,71.333,-156.65,2750.4353299559803
690020,93218,69002093218,36.0,-121.233,70045027512,LONELY,AK,LNI,70.917,-153.25,2676.3554370627153
690020,93218,69002093218,36.0,-121.233,70063027403,OLIKTOK POW 2,AK,POLI,70.5,-149.883,2604.050248854232
690020,93218,69002093218,36.0,-121.233,70063526465,GALBRAITH LAKE AIRPORT,AK,PAGB,68.479,-149.49,2490.9756094472277
690020,93218,69002093218,36.0,-121.233,70063627405,PRUDHOE BAY,AK,PAUD,70.25,-148.333,2568.180281844432
690020,93218,69002093218,36.0,-121.233,70104626418,CENTRAL AIRPORT,AK,PACE,65.567,-144.765,2254.5584891291946
690020,93218,69002093218,36.0,-121.233,70119526625,SHISHMAREF/NEW AIRPORT,AK,PASH,66.25,-166.089,2743.6881828292408


In [0]:
display(df_stations.describe())

summary,usaf,wban,station_id,lat,lon,neighbor_id,neighbor_name,neighbor_state,neighbor_call,neighbor_lat,neighbor_lon,distance_to_neighbor
count,5004169,5004169,5004169,5004169,5004169,5004169,5004169,5004169,5004169,5004169,5004169,5004169
mean,724806.8676069154,32518.517210549842,7.248071938615863E10,39.835810907459745,-98.71206392527364,7.248071938584857E10,null,null,null,39.835810907559264,-98.712063925185,1343.5152928146138
stddev,17516.71784985392,33231.79213586752,1.7516719941563978E9,8.602171852773631,22.15222249957667,1.7516719941555088E9,null,null,null,8.602171852773466,22.152222499577764,948.8454195513482
min,690020,00102,69002093218,17.7,-176.65,69002093218,A L MANGHAM JR RGNL ARPT,AK,4DG,17.7,-176.65,0.0
max,A51256,96402,A5125600451,71.333,174.1,A5125600451,ZEPHYRHILLS MUNICIPAL AIRPORT,WY,TSG,71.333,174.1,6435.969681007664


In [0]:
df_stations.printSchema()

root
-- usaf: string (nullable = true)
-- wban: string (nullable = true)
-- station_id: string (nullable = true)
-- lat: double (nullable = true)
-- lon: double (nullable = true)
-- neighbor_id: string (nullable = true)
-- neighbor_name: string (nullable = true)
-- neighbor_state: string (nullable = true)
-- neighbor_call: string (nullable = true)
-- neighbor_lat: double (nullable = true)
-- neighbor_lon: double (nullable = true)
-- distance_to_neighbor: double (nullable = true)

#Airlines Data Transformation

In [0]:
# drop diverted flight info
airlines_cols_keep = ['YEAR','QUARTER','MONTH','DAY_OF_MONTH', 'DAY_OF_WEEK','FL_DATE','OP_UNIQUE_CARRIER','OP_CARRIER_AIRLINE_ID','OP_CARRIER', 'TAIL_NUM','OP_CARRIER_FL_NUM','ORIGIN_AIRPORT_ID','ORIGIN_AIRPORT_SEQ_ID','ORIGIN_CITY_MARKET_ID','ORIGIN','ORIGIN_CITY_NAME','ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS','ORIGIN_STATE_NM','ORIGIN_WAC','DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR', 'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_WAC', 'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15', 'DEP_DELAY_GROUP', 'DEP_TIME_BLK', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'ARR_DELAY_NEW', 'ARR_DEL15', 'ARR_DELAY_GROUP', 'ARR_TIME_BLK', 'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'FLIGHTS', 'DISTANCE', 'DISTANCE_GROUP', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']

airlines = df_airlines.select(*airlines_cols_keep)

# create a new var for delayed + cancelled due to weather
#airlines = df_airlines
# create an unique flight ID for flights that have either departure or cancelled info
airlines = airlines.filter(f.col('DEP_TIME').isNotNull() | f.col('CANCELLED').isNotNull()) \
                   .withColumn('flightID', f.monotonically_increasing_id())

In [0]:
# Clean null values if departure is on time
def replace(column, ref_column, CRS_column):
    col = f.when((ref_column == CRS_column) & (column == f.lit(None)), 0).otherwise(column)
    return col

airlines = airlines.withColumn('DEP_DEL15', replace(f.col('DEP_DEL15'), f.col('DEP_TIME'), f.col('CRS_DEP_TIME')))

In [0]:
print('the number of records in airline data is ', airlines.count(), 'and the number of attributes is ', len(airlines_cols_keep)+1)

the number of records in airline data is 31746841 and the number of attributes is 62

2015: the number of records in airline data is  5819079 and the number of attributes is  62

2016: the number of records in airline data is  5617658 and the number of attributes is  62

2017: the number of records in airline data is  5674621 and the number of attributes is  62

2018: the number of records in airline data is  7213446 and the number of attributes is  62

In [0]:
# Remove cancelled flights from train data
airlines = airlines.where(f.col("CANCELLED") != 1)

In [0]:
# Remove diverted flights
airlines = airlines.where(f.col("DIVERTED") != 1)

In [0]:
print('the number of records in airline data is ', airlines.count())

the number of records in airline data is 31178801

2015: the number of records in airline data is  5714008

2016: the number of records in airline data is  5538145

2017: the number of records in airline data is  5579411

2018: the number of records in airline data is  7079005

### Join airline dataset with external OpenFlight data

In [0]:
# Get airport database from OpenFlights https://openflights.org/data.html
airport_data = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat',header=None)
airport_data = airport_data.rename(columns= {0:'AirportID',1:'Name',2:'City',3:'Country',4:'IATA',5:'ICAO',6:'Latitude',7:'Longitude', 8:'Altitude',9:'Timezone',10:'DST',11:'TZ_Timezone',12:'Type',13:'Source'})
df_airport = spark.createDataFrame(airport_data)

In [0]:
# Add timezone and airport ICAO code to airline data

# Join on origin airport
df_airlines_airport = airlines.join(df_airport, airlines.ORIGIN == df_airport.IATA, how='left').select(airlines['*'], df_airport['AirportID'].alias('ORIGIN_airportID'), df_airport['ICAO'].alias('ORIGIN_ICAO'), df_airport['Timezone'].alias('ORIGIN_Timezone'), df_airport['TZ_Timezone'].alias('ORIGIN_TZ'))

# Join on destination airport
df_airlines_airport = df_airlines_airport.join(df_airport, df_airlines_airport.DEST == df_airport.IATA, how='left').select(df_airlines_airport['*'], df_airport['AirportID'].alias('DEST_airportID'), df_airport['ICAO'].alias('DEST_ICAO'), df_airport['Timezone'].alias('DEST_Timezone'), df_airport['TZ_Timezone'].alias('DEST_TZ'))

In [0]:
display(df_airlines_airport.limit(50))
print('the number of records in airline data is ', df_airlines_airport.count(), 'and the number of attributes is ', len(df_airlines_airport.columns))

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,flightID,ORIGIN_airportID,ORIGIN_ICAO,ORIGIN_Timezone,ORIGIN_TZ,DEST_airportID,DEST_ICAO,DEST_Timezone,DEST_TZ
2019,2,6,12,3,2019-06-12,OO,20304,OO,N468CA,7400,10577,1057705,30577,BGM,"Binghamton, NY",NY,36,New York,22,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,600,550,-10.0,0.0,0.0,-1,0600-0659,14.0,604,705,7.0,733,712,-21.0,0.0,0.0,-2,0700-0759,0.0,null,0.0,93.0,82.0,61.0,1.0,378.0,2,null,null,null,null,null,35169,4129,KBGM,-5,America/New_York,3645,KDTW,-5,America/New_York
2019,2,6,12,3,2019-06-12,OO,20304,OO,N875AS,7420,10577,1057705,30577,BGM,"Binghamton, NY",NY,36,New York,22,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1755,1751,-4.0,0.0,0.0,-1,1700-1759,6.0,1757,1902,11.0,1924,1913,-11.0,0.0,0.0,-1,1900-1959,0.0,null,0.0,89.0,82.0,65.0,1.0,378.0,2,null,null,null,null,null,35195,4129,KBGM,-5,America/New_York,3645,KDTW,-5,America/New_York
2019,2,6,12,3,2019-06-12,OO,20304,OO,N426SW,7435,10577,1057705,30577,BGM,"Binghamton, NY",NY,36,New York,22,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1300,1254,-6.0,0.0,0.0,-1,1300-1359,9.0,1303,1409,11.0,1429,1420,-9.0,0.0,0.0,-1,1400-1459,0.0,null,0.0,89.0,86.0,66.0,1.0,378.0,2,null,null,null,null,null,35212,4129,KBGM,-5,America/New_York,3645,KDTW,-5,America/New_York
2019,2,6,1,6,2019-06-01,OO,20304,OO,N433SW,7400,10577,1057705,30577,BGM,"Binghamton, NY",NY,36,New York,22,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,915,911,-4.0,0.0,0.0,-1,0900-0959,10.0,921,1023,9.0,1049,1032,-17.0,0.0,0.0,-2,1000-1059,0.0,null,0.0,94.0,81.0,62.0,1.0,378.0,2,null,null,null,null,null,36258,4129,KBGM,-5,America/New_York,3645,KDTW,-5,America/New_York
2019,2,6,2,7,2019-06-02,OO,20304,OO,N868CA,7400,10577,1057705,30577,BGM,"Binghamton, NY",NY,36,New York,22,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,600,554,-6.0,0.0,0.0,-1,0600-0659,13.0,607,721,10.0,735,731,-4.0,0.0,0.0,-1,0700-0759,0.0,null,0.0,95.0,97.0,74.0,1.0,378.0,2,null,null,null,null,null,38575,4129,KBGM,-5,America/New_York,3645,KDTW,-5,America/New_York
2019,2,6,2,7,2019-06-02,OO,20304,OO,N433SW,7420,10577,1057705,30577,BGM,"Binghamton, NY",NY,36,New York,22,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1754,1747,-7.0,0.0,0.0,-1,1700-1759,18.0,1805,1913,9.0,1929,1922,-7.0,0.0,0.0,-1,1900-1959,0.0,null,0.0,95.0,95.0,68.0,1.0,378.0,2,null,null,null,null,null,38599,4129,KBGM,-5,America/New_York,3645,KDTW,-5,America/New_York
2019,2,6,2,7,2019-06-02,OO,20304,OO,N461SW,7435,10577,1057705,30577,BGM,"Binghamton, NY",NY,36,New York,22,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1315,1422,67.0,67.0,1.0,4,1300-1359,14.0,1436,1543,14.0,1448,1557,69.0,69.0,1.0,4,1400-1459,0.0,null,0.0,93.0,95.0,67.0,1.0,378.0,2,67.0,0.0,2.0,0.0,0.0,38616,4129,KBGM,-5,America/New_York,3645,KDTW,-5,America/New_York
2019,2,6,17,1,2019-06-17,OO,20304,OO,N953SW,7400,10577,1057705,30577,BGM,"Binghamton, NY",NY,36,New York,22,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,600,554,-6.0,0.0,0.0,-1,0600-0659,14.0,608,717,17.0,733,734,1.0,1.0,0.0,0,0700-0759,0.0,null,0.0,93.0,100.0,69.0,1.0,378.0,2,null,null,null,null,null,83833,4129,KBGM,-5,America/New_York,3645,KDTW,-5,America/New_York
2019,2,6,17,1,2019-0

the number of records in airline data is 31178801 and the number of attributes is 70

In [0]:
display(df_airlines_airport.describe())

summary,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,DIV_ARR_DELAY,DIV_DISTANCE,DIV1_AIRPORT,DIV1_AIRPORT_ID,DIV1_AIRPORT_SEQ_ID,DIV1_WHEELS_ON,DIV1_TOTAL_GTIME,DIV1_LONGEST_GTIME,DIV1_WHEELS_OFF,DIV1_TAIL_NUM,DIV2_AIRPORT,DIV2_AIRPORT_ID,DIV2_AIRPORT_SEQ_ID,DIV2_WHEELS_ON,DIV2_TOTAL_GTIME,DIV2_LONGEST_GTIME,DIV2_WHEELS_OFF,DIV2_TAIL_NUM,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV3_AIRPORT_SEQ_ID,DIV3_WHEELS_ON,DIV3_TOTAL_GTIME,DIV3_LONGEST_GTIME,DIV3_WHEELS_OFF,DIV3_TAIL_NUM,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV4_AIRPORT_SEQ_ID,DIV4_WHEELS_ON,DIV4_TOTAL_GTIME,DIV4_LONGEST_GTIME,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM,ORIGIN_airportID,ORIGIN_ICAO,ORIGIN_Timezone,ORIGIN_TZ,DEST_airportID,DEST_ICAO,DEST_Timezone,DEST_TZ
count,161057,161057,161057,161057,161057,161057,161057,161057,161057,160375,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,156231,156231,156231,156231,156231,161057,156121,156121,155957,155957,161057,155957,155692,155692,155692,155692,161057,161057,4984,161057,161057,155692,155692,161057,161057,161057,34915,34915,34915,34915,34915,1053,1053,1053,161057,381,265,265,381,429,429,429,429,429,429,278,278,17,17,17,17,17,17,5,5,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,161057,161057,161057,161057,161057,161057,161057,161057
mean,2015.0,1.0,2.0365522765232185,15.591504870946311,3.9165202381765463,null,null,20010.91815320042,null,null,2508.798711015355,11963.80259162905,1196384.3722098388,30654.21638922866,null,null,null,14.773906132611435,null,37.10433573207001,12707.705110613013,1270773.0347454627,32018.14314807801,null,null,null,28.618718838672024,null,49.025475452790005,1422.252767653688,1437.2168967746477,12.583827793459685,14.368870454647285,0.2348125532064699,0.2524274951834143,null,18.062086458580204,1465.591502744666,1563.5781721884878,6.14908596600345,1584.805360834984,1566.3260642356547,7.511567710608124,14.163483030598874,0.22425686612028878,0.00872877219124939,null,0.030945565855566664,null,0.0023656221089427966,124.43478395847433,120.03714384811038,95.8361765537086,1.0,662.2313404571053,3.1245459681975944,19.903050264929114,6.769039094944866,12.16838035228412,0.017556923958184163,19.076671917513963,1399.9924026590693,36.22602089268756,35.970560303893635,0.005116201096506206,0.6955380577427821,347.10188679245283,232.63396226415094,47.90813648293963,null,12499.242424242424,1249927.13986014,1464.04662004662,27.405594405594407,22.242424242424242,1474.8848920863309,null,null,12674.176470588236,1267420.8823529412,1341.4117647058824,11.941176470588236,10.235294117647058,1392.2,null,null,10397.0,1039705.0,51.0,4.0,4.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3747.6345269066233,null,-5.443476533152859,null,3779.3610585072365,null,-5.663013715641046,null
stddev,0.0,0.0,0.8326601487890224,8.67952791653968,1.9958147997611477,null,null,314.6264968572746,null,null,1746.4192642359562,1755.1815640743703,175517.1628147043,28

In [0]:
# Time conversion
# convert departure time to timestamp
# create a new field for CRS departure time - 2 hours
df_airlines_airport = df_airlines_airport \
                     .withColumn('CRS_DEP_DATETIME', f.to_timestamp(f.concat(f.col('FL_DATE'), f.lpad(f.col('CRS_DEP_TIME'), 4, '0')), 
                                                                    format='yyyy-MM-ddHHmm')) \
                     .withColumn('ACT_DEP_DATETIME', f.to_timestamp(f.concat(f.col('FL_DATE'), f.lpad(f.col('DEP_TIME'), 4, '0')), 
                                                                    format='yyyy-MM-ddHHmm')) \
                     .withColumn('EARLIER_DATETIME', (f.unix_timestamp("CRS_DEP_DATETIME") - 2*3600).cast('timestamp')) \
                     .withColumn('CRS_DEP_DATETIME', f.to_utc_timestamp(f.col('CRS_DEP_DATETIME'), f.col('ORIGIN_TZ'))) \
                     .withColumn('ACT_DEP_DATETIME', f.to_utc_timestamp(f.col('ACT_DEP_DATETIME'), f.col('ORIGIN_TZ'))) \
                     .withColumn('EARLIER_DATETIME', f.to_utc_timestamp(f.col('EARLIER_DATETIME'), f.col('ORIGIN_TZ'))) \

# convert ARRIVAL time to timestamp
df_airlines_airport = df_airlines_airport \
                     .withColumn('CRS_ARR_DATETIME', f.to_timestamp(f.concat(f.col('FL_DATE'), f.lpad(f.col('CRS_ARR_TIME'), 4, '0')), 
                                                                    format='yyyy-MM-ddHHmm')) \
                     .withColumn('ACT_ARR_DATETIME', f.to_timestamp(f.concat(f.col('FL_DATE'), f.lpad(f.col('ARR_TIME'), 4, '0')), 
                                                                    format='yyyy-MM-ddHHmm')) \
                     .withColumn('CRS_ARR_DATETIME', f.to_utc_timestamp(f.col('CRS_ARR_DATETIME'), f.col('DEST_TZ'))) \
                     .withColumn('ACT_ARR_DATETIME', f.to_utc_timestamp(f.col('ACT_ARR_DATETIME'), f.col('DEST_TZ'))) 

#df_airlines_airport = df_airlines_airport.where()
#display(df_airlines_airport.limit(50))

### Save to Cloud storage so we don't need to rerun the above process when re-start the cluster.

In [0]:
# save transformed airline data
df_airlines_airport.write.mode('overwrite').parquet(f"/mnt/team11/airlines_" + timeframe + "transformed")

### Read from Cloud storage

In [0]:
df_airlines_airport = spark.read.parquet("/mnt/team11/airlines_" + timeframe + "transformed")

In [0]:
display(df_airlines_airport.limit(50))

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,flightID,ORIGIN_airportID,ORIGIN_ICAO,ORIGIN_Timezone,ORIGIN_TZ,DEST_airportID,DEST_ICAO,DEST_Timezone,DEST_TZ,CRS_DEP_DATETIME,ACT_DEP_DATETIME,EARLIER_DATETIME,CRS_ARR_DATETIME,ACT_ARR_DATETIME
2015,1,2,1,7,2015-02-01,AA,19805,AA,N016AA,349,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,2105,2116,11.0,11.0,0.0,0,2100-2159,12.0,2128,2252,9.0,2300,2301,1.0,1.0,0.0,0,2300-2359,0.0,null,0.0,115.0,105.0,84.0,1.0,594.0,3,null,null,null,null,null,247,3682,KATL,-5,America/New_York,3576,KMIA,-5,America/New_York,2015-02-02T02:05:00.000+0000,2015-02-02T02:16:00.000+0000,2015-02-02T00:05:00.000+0000,2015-02-02T04:00:00.000+0000,2015-02-02T04:01:00.000+0000
2015,1,2,2,1,2015-02-02,AA,19805,AA,N019AA,349,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,2105,2056,-9.0,0.0,0.0,-1,2100-2159,24.0,2120,2242,6.0,2300,2248,-12.0,0.0,0.0,-1,2300-2359,0.0,null,0.0,115.0,112.0,82.0,1.0,594.0,3,null,null,null,null,null,248,3682,KATL,-5,America/New_York,3576,KMIA,-5,America/New_York,2015-02-03T02:05:00.000+0000,2015-02-03T01:56:00.000+0000,2015-02-03T00:05:00.000+0000,2015-02-03T04:00:00.000+0000,2015-02-03T03:48:00.000+0000
2015,1,2,3,2,2015-02-03,AA,19805,AA,N008AA,349,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,2105,2058,-7.0,0.0,0.0,-1,2100-2159,19.0,2117,2243,7.0,2300,2250,-10.0,0.0,0.0,-1,2300-2359,0.0,null,0.0,115.0,112.0,86.0,1.0,594.0,3,null,null,null,null,null,249,3682,KATL,-5,America/New_York,3576,KMIA,-5,America/New_York,2015-02-04T02:05:00.000+0000,2015-02-04T01:58:00.000+0000,2015-02-04T00:05:00.000+0000,2015-02-04T04:00:00.000+0000,2015-02-04T03:50:00.000+0000
2015,1,2,4,3,2015-02-04,AA,19805,AA,N024AA,349,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,2105,2057,-8.0,0.0,0.0,-1,2100-2159,23.0,2120,2245,3.0,2300,2248,-12.0,0.0,0.0,-1,2300-2359,0.0,null,0.0,115.0,111.0,85.0,1.0,594.0,3,null,null,null,null,null,250,3682,KATL,-5,America/New_York,3576,KMIA,-5,America/New_York,2015-02-05T02:05:00.000+0000,2015-02-05T01:57:00.000+0000,2015-02-05T00:05:00.000+0000,2015-02-05T04:00:00.000+0000,2015-02-05T03:48:00.000+0000
2015,1,2,6,5,2015-02-06,AA,19805,AA,N006AA,349,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,2105,2057,-8.0,0.0,0.0,-1,2100-2159,25.0,2122,2242,5.0,2300,2247,-13.0,0.0,0.0,-1,2300-2359,0.0,null,0.0,115.0,110.0,80.0,1.0,594.0,3,null,null,null,null,null,251,3682,KATL,-5,America/New_York,3576,KMIA,-5,America/New_York,2015-02-07T02:05:00.000+0000,2015-02-07T01:57:00.000+0000,2015-02-07T00:05:00.000+0000,2015-02-07T04:00:00.000+0000,2015-02-07T03:47:00.000+0000
2015,1,2,7,6,2015-02-07,AA,19805,AA,N012AA,349,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,2105,2058,-7.0,0.0,0.0,-1,2100-2159,12.0,2110,2240,8.0,2300,2248,-12.0,0.0,0.0,-1,2300-2359,0.0,null,0.0,115.0,110.0,90.0,1.0,594.0,3,null,null,null,null,null,252,3682,KATL,-5,America/New_York,3576,KMIA,-5,America/New_York,2015-02-08

In [0]:
display(df_weather.limit(50))

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AW1,GA1,GA2,GA3,GA4,GE1,GF1,KA1,KA2,MA1,MD1,MW1,MW2,OC1,OD1,OD2,REM,EQD,AW2,AX4,GD1,AW5,GN1,AJ1,AW3,MK1,KA4,GG3,AN1,RH1,AU5,HL1,OB1,AT8,AW7,AZ1,CH1,RH3,GK1,IB1,AX1,CT1,AK1,CN2,OE1,MW5,AO1,KA3,AA3,CR1,CF2,KB2,GM1,AT5,AY2,MW6,MG1,AH6,AU2,GD2,AW4,MF1,AA1,AH2,AH3,OE3,AT6,AL2,AL3,AX5,IB2,AI3,CV3,WA1,GH1,KF1,CU2,CT3,SA1,AU1,KD2,AI5,GO1,GD3,CG3,AI1,AL1,AW6,MW4,AX6,CV1,ME1,KC2,CN1,UA1,GD5,UG2,AT3,AT4,GJ1,MV1,GA5,CT2,CG2,ED1,AE1,CO1,KE1,KB1,AI4,MW3,KG2,AA2,AX2,AY1,RH2,OE2,CU3,MH1,AM1,AU4,GA6,KG1,AU3,AT7,KD1,GL1,IA1,GG2,OD3,UG1,CB1,AI6,CI1,CV2,AZ2,AD1,AH1,WD1,AA4,KC1,IA2,CF3,AI2,AT1,GD4,AX3,AH4,KB3,CU1,CN4,AT2,CG1,CF1,GG1,MV2,CW1,GG4,AB1,AH5,CN3
3809099999,2015-01-01T00:00:00.000+0000,4,50.086092,-5.255711,81.38,"CULDROSE, UK",FM-12,99999,V020,"200,1,N,0077,1","00240,1,C,N","008000,1,9,9","+0113,1","+0099,1","10306,1",,"01,1,+00180,1,07,1","05,1,+00240,1,07,1","08,1,+00360,1,07,1",,"9,AGL ,+99999,+99999","08,99,1,01,1,99,9,00180,1,99,9,99,9",,,"99999,9,10213,1","3,1,002,1,+999,9","51,1",,,"3,99,0144,1,999","4,99,0134,1,999",SYN10603809 11358 82015 10113 20099 30213 40306 53002 69901 75165 887// 333 81706 85708 88712 90710 91128 91026=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"5,1,02,1",,,,,,,,"06,0000,2,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"6,1,02,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3809099999,2015-01-01T00:50:00.000+0000,4,50.086092,-5.255711,81.38,"CULDROSE, UK",FM-15,99999,V020,"210,1,N,0077,1","00183,1,C,N","008000,1,9,9","+0110,1","+0100,1","99999,9",,"02,1,+00122,1,99,9","04,1,+00183,1,99,9","08,1,+00305,1,99,9",,"9,AGL ,+99999,+99999","99,99,9,02,1,99,9,00122,1,99,9,99,9",,,"10290,1,99999,9",,"51,1",,,,,MET079METAR EGDR 010050Z 21015KT 8000 -DZ FEW004 SCT006 OVC010 11/10 Q1029 YLO1=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3809099999,2015-01-01T01:00:00.000+0000,4,50.086092,-5.255711,81.38,"CULDROSE, UK",FM-12,99999,V020,"210,1,N,0077,1","00300,1,9,N","008000,1,9,9","+0113,1","+0101,1","10300,1",,"01,1,+00120,1,07,1","03,1,+00180,1,07,1","08,1,+00300,1,07,1",,"9,AGL ,+99999,+99999","08,99,1,01,1,99,9,00120,1,99,9,99,9",,,"99999,9,10206,1","8,1,004,1,+999,9","51,1",,,"3,99,0134,1,999",,SYN09403809 41258 82115 10113 20101 30206 40300 58004 75155 887// 333 81704 83706 88710 90710 91126=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"5,1,02,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"5,1,02,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3809099999,2015-01-01T01:50:00.000+0000,4,50.086092,-5.255711,81.38,"CULDROSE, UK",FM-15,99999,V020,"200,1,N,0082,1","00244,1,9,N","008000,1,9,9","+0120,1","+0100,1","99999,9",,"04,1,+00183,1,99,9","07,1,+00244,1,99,9","08,1,+00305,1,99,9",,"9,AGL ,+99999,+99999","99,99,9,04,1,99,9,00183,1,99,9,99,9",,,"10290,1,99999,9",,"05,1",,"0144,1",,,MET086METAR EGDR 010150Z 20016G28KT 8000 HZ SCT006 BKN008 OVC010 12/10 Q1029 REDZ YLO1=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3809099999,2015-01-01T02:00:00.000+0000,4,50.086092,-5.255711,81.38,"CULDROSE, UK",FM-12,99999,V020,"200,1,N,0082,1","00240,1,C,N","008000,1,9,9","+0115,1","+0100,1","10294,1",,"03,1,+00180,1,07,1","05,1,+00240,1,07,1","08,1,+00300,1,07,1",,"9,AGL ,+99999,+99999","08,99,1,03,1,99,9,00180,1,99,9,99,9",,,"99999,9,10201,1","8,1,008,1,+999,9","20,1",,,"3,99,0149,1,999","4,99,0144,1,999",SYN10003809 41358 82016 10115 20100 30201 40294 58008 72052 886// 333 83706 85708 88710 90710 91129 91028=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"2,1,02,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"5,1,02,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3809099999,2015-01-01T02:50:00.000+0000,4,50.086092,-5.255711,81.38,"CULDROSE, UK",FM-15,99999,V020,"210,1,N,0093,1","00122,1,9,N","006000,1

#Weather Data Transformation

In [0]:
# Weather data transformation

df_weather = df_weather.withColumn("direction_angle", f.split(f.col("WND"),",").getItem(0).cast(IntegerType())) \
             .withColumn("direction_quality", f.split(f.col("WND"),",").getItem(1).cast(IntegerType())) \
             .withColumn("type", f.split(f.col("WND"),",").getItem(2)) \
             .withColumn("speed", f.split(f.col("WND"),",").getItem(3).cast(IntegerType())) \
             .withColumn("speed_quality", f.split(f.col("WND"),",").getItem(4).cast(IntegerType())) \
             .withColumn("ceiling_height", f.split(f.col("CIG"),",").getItem(0).cast(IntegerType())) \
             .withColumn("ceiling_quality", f.split(f.col("CIG"),",").getItem(1).cast(IntegerType())) \
             .withColumn("ceiling_determination", f.split(f.col("CIG"),",").getItem(2)) \
             .withColumn("CAVOK", f.split(f.col("CIG"),",").getItem(3)) \
             .withColumn("vis_distance", f.split(f.col("VIS"),",").getItem(0).cast(IntegerType())) \
             .withColumn("distance_quality", f.split(f.col("VIS"),",").getItem(1).cast(IntegerType())) \
             .withColumn("variability", f.split(f.col("VIS"),",").getItem(2)) \
             .withColumn("quality_variability", f.split(f.col("VIS"),",").getItem(3).cast(IntegerType())) \
             .withColumn("air_temp", f.split(f.col("TMP"),",").getItem(0).cast(IntegerType())) \
             .withColumn("air_temp_quality", f.split(f.col("TMP"),",").getItem(1)) \
             .withColumn("dew_point", f.split(f.col("DEW"),",").getItem(0).cast(IntegerType())) \
             .withColumn("dew_point_quality", f.split(f.col("DEW"),",").getItem(1)) \
             .withColumn("sea_level_pressure", f.split(f.col("SLP"),",").getItem(0).cast(IntegerType())) \
             .withColumn("sea_level_pressure_quality", f.split(f.col("SLP"),",").getItem(1).cast(IntegerType())) \
             .withColumn("precipitation_hrs", f.split(f.col("AA1"),",").getItem(0).cast(IntegerType())) \
             .withColumn("precipitation_depth", f.split(f.col("AA1"),",").getItem(1).cast(IntegerType())) \
             .withColumn("precipitation_quality", f.split(f.col("AA1"),",").getItem(3))

#display(df_weather.limit(100))

In [0]:
display(df_stations.limit(100))

usaf,wban,station_id,lat,lon,neighbor_id,neighbor_name,neighbor_state,neighbor_call,neighbor_lat,neighbor_lon,distance_to_neighbor
690020,93218,69002093218,36.0,-121.233,69002093218,JOLON HUNTER LIGGETT MIL RES,CA,KHGT,36.0,-121.233,0.0
690020,93218,69002093218,36.0,-121.233,69007093217,FRITZSCHE AAF,CA,KOAR,36.683,-121.767,55.73024537916726
690020,93218,69002093218,36.0,-121.233,69014093101,EL TORO MCAS,CA,KNZJ,33.667,-117.733,255.49106220353934
690020,93218,69002093218,36.0,-121.233,70027127506,BARROW POINT BARROW,AK,KPBA,71.333,-156.65,2750.4353299559803
690020,93218,69002093218,36.0,-121.233,70045027512,LONELY,AK,LNI,70.917,-153.25,2676.3554370627153
690020,93218,69002093218,36.0,-121.233,70063027403,OLIKTOK POW 2,AK,POLI,70.5,-149.883,2604.050248854232
690020,93218,69002093218,36.0,-121.233,70063526465,GALBRAITH LAKE AIRPORT,AK,PAGB,68.479,-149.49,2490.9756094472277
690020,93218,69002093218,36.0,-121.233,70063627405,PRUDHOE BAY,AK,PAUD,70.25,-148.333,2568.180281844432
690020,93218,69002093218,36.0,-121.233,70104626418,CENTRAL AIRPORT,AK,PACE,65.567,-144.765,2254.5584891291946
690020,93218,69002093218,36.0,-121.233,70119526625,SHISHMAREF/NEW AIRPORT,AK,PASH,66.25,-166.089,2743.6881828292408


In [0]:
# columns to keep
wth_fields = ['STATION','DATE','LATITUDE','LONGITUDE','ELEVATION','NAME','direction_angle', 'direction_quality', 'type', 'speed','speed_quality','ceiling_height','ceiling_quality','ceiling_determination','CAVOK','vis_distance', 'distance_quality', 'variability', 'quality_variability','air_temp','air_temp_quality', 'dew_point','dew_point_quality','sea_level_pressure', 'sea_level_pressure_quality', 'precipitation_hrs', 'precipitation_depth', 'precipitation_quality']
stt_fields = ['station_id', 'neighbor_name','neighbor_state','neighbor_call','distance_to_neighbor']
df_weather_select = df_weather.select(wth_fields)
df_weather_select = df_weather_select.dropDuplicates(['STATION','DATE'])

In [0]:
print(df_weather_select.count())

600383907

In [0]:
display(df_weather_select.describe())

summary,STATION,LATITUDE,LONGITUDE,ELEVATION,NAME,direction_angle,direction_quality,type,speed,speed_quality,ceiling_height,ceiling_quality,ceiling_determination,CAVOK,vis_distance,distance_quality,variability,quality_variability,air_temp,air_temp_quality,dew_point,dew_point_quality,sea_level_pressure,sea_level_pressure_quality,precipitation_hrs,precipitation_depth,precipitation_quality
count,28536449,28570121,28570121,28570121,28570121,28570121,28570097,28570121,28570121,28569543,28570121,28570121,28570121,28570121,28570121,28560657,28570121,28551608,28570121,28570121,28570121,28570121,28570121,28570121,5990417,5990417,5990417
mean,5.899198141182356E10,38.135372357283416,-37.960003633584755,369.3218384221008,null,420.5400085634919,4.25218363801845,9.0,1583.179384154516,3.5256649712597783,53188.543924017686,6.128260604846581,9.0,9.0,332775.9730944437,4.842156957383718,9.0,7.723983601904313,191.0785946968863,2.2926336459580945,1592.6969007936648,3.4970497649858743,68059.24611610851,6.479792297694504,3.984321959556405,480.059739747667,2.8849454262301655
stddev,3.247016747350616E10,21.187877112343852,79.34993866906233,539.1305389354501,null,370.39145562616494,3.3699255873177756,0.0,3606.4550782069014,2.9476171719077473,45750.45184290233,3.1063953597399907,0.0,0.0,461455.8501119221,3.306395546190651,0.0,1.8643778064748526,1225.509000875383,2.0037358665300227,3671.1745049116153,3.019413546842917,43000.557674145886,3.5372965266120215,7.308321855035509,2126.2204889881086,2.3431153077280564
min,10000199999,-90.0,-179.9833333,-999.9,,1,1,9,0,1,0,1,9,9,0,1,9,4,-858,1,-800,1,9309,1,0,0,0
max,A5125600451,83.65,179.75,4701.0,"ZYRYANKA, RS",999,9,V,9999,9,99999,9,W,Y,999999,9,V,9,9999,P,9999,P,99999,9,99,9999,U


### Weather data clean
1. Remove missing value representations "999*" and replace with Null
2. Replace values with Null if the quality codes are suspect/erroneous (2,3,6,7)

In [0]:
def replace(column, value, qualityCol, qualityList):
    col = f.when(column != value, column).otherwise(f.lit(None))
    col = f.when(qualityCol.isin(qualityList), f.lit(None)).otherwise(col)
    return col

df_weather_select = df_weather_select.withColumn('direction_angle', replace(f.col('direction_angle'), 999, f.col('direction_quality'), [2,3,6,7])) \
                   .withColumn('speed', replace(f.col('speed'), 9999, f.col('speed_quality'), [2,3,6,7])) \
                   .withColumn('ceiling_height', replace(f.col('ceiling_height'), 99999, f.col('ceiling_quality'), [2,3,6,7])) \
                   .withColumn('vis_distance', replace(f.col('vis_distance'), 999999, f.col('distance_quality'), [2,3,6,7])) \
                   .withColumn('variability', replace(f.col('variability'), 999, f.col('quality_variability'), [2,3,6,7])) \
                   .withColumn('air_temp', replace(f.col('air_temp'), 9999, f.col('air_temp_quality'), ['2','3','6','7'])) \
                   .withColumn('dew_point', replace(f.col('dew_point'), 9999, f.col('dew_point_quality'), ['2','3','6','7'])) \
                   .withColumn('sea_level_pressure', replace(f.col('sea_level_pressure'), 99999, f.col('sea_level_pressure_quality'), [2,3,6,7])) \
                   .withColumn('precipitation_hrs', replace(f.col('precipitation_hrs'), 99, f.col('precipitation_quality'), [2,3,6,7])) \
                   .withColumn('precipitation_depth', replace(f.col('precipitation_depth'), 9999, f.col('precipitation_quality'), [2,3,6,7]))

In [0]:
display(df_weather_select.describe())

summary,STATION,LATITUDE,LONGITUDE,ELEVATION,NAME,direction_angle,direction_quality,type,speed,speed_quality,ceiling_height,ceiling_quality,ceiling_determination,CAVOK,vis_distance,distance_quality,variability,quality_variability,air_temp,air_temp_quality,dew_point,dew_point_quality,sea_level_pressure,sea_level_pressure_quality,precipitation_hrs,precipitation_depth,precipitation_quality
count,28536449,28570121,28570121,28570121,28570121,20602735,28570097,28570121,24135518,28569543,14771625,28570121,28570121,28570121,19299636,28560657,19443609,28551608,28080282,28570121,23969382,28570121,10147004,28570121,5948874,5679345,5988795
mean,5.8991981411823586E10,38.1376206159495,-37.96007824986482,369.3443142037051,null,196.82684609591882,4.252119655036523,9.0,37.78481800142015,3.5256645512320586,9606.749773366166,6.128243804077694,9.0,9.0,13730.934214717832,4.8421573775421205,9.0,7.723983601904313,39.66378738646571,2.2926292578047502,-9.106996917984786,3.4970598226224214,10167.793768584303,6.479883931888143,3.6861972534634284,4.830994243174169,2.8843708559239625
stddev,3.247016747350617E10,21.18775141907517,79.35028560504573,539.0861094218745,null,103.85957416344071,3.3698734149792298,0.0,31.61669322114682,2.947616676824024,10209.630949118226,3.106390644936117,0.0,0.0,9028.838106947,3.3063956191961275,0.0,1.8643778064748526,126.9887258180629,2.0037251291032994,120.48373893529723,3.0194253205938644,117.64504407058475,3.5373102092524946,5.453715669354921,31.09819917870345,2.343172750100585
min,10000199999,-90.0,-179.9833333,-999.9,,1,1,9,0,1,0,1,9,9,0,1,9,4,-744,1,-800,1,9309,1,0,0,0
max,A5125600451,83.65,179.75,4701.0,"ZYRYANKA, RS",360,9,V,490,9,22000,9,W,Y,160000,9,9,9,540,P,368,P,10899,9,24,4998,U


### Find closest neighbors for all weather stations

In [0]:
# for each weather station, find the minimal distance to neighbor
from pyspark.sql import Window
w = Window.partitionBy('neighbor_call')
df_station_processed = df_stations.withColumn('min_distance_to_neighbor', f.min('distance_to_neighbor').over(w))\
    .where(f.col('distance_to_neighbor') == f.col('min_distance_to_neighbor'))\
    .drop('min_distance_to_neighbor')


In [0]:
# Every airport has a weather station?
display(df_station_processed.where(f.col('min_distance_to_neighbor')>0))

usaf,wban,station_id,lat,lon,neighbor_id,neighbor_name,neighbor_state,neighbor_call,neighbor_lat,neighbor_lon,distance_to_neighbor


### Weather+airline join

In [0]:
# attach the nearest weather station to origin airport
df_airlines_airport_station_processed = df_airlines_airport.join(df_station_processed, 
                                                                 df_airlines_airport.ORIGIN_ICAO == df_station_processed.neighbor_call, 'left') \
                                       .select(df_airlines_airport['*'], df_station_processed['station_id'].alias('ORIGIN_near_station'))


In [0]:
display(df_airlines_airport_station_processed.limit(10))

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,flightID,ORIGIN_airportID,ORIGIN_ICAO,ORIGIN_Timezone,ORIGIN_TZ,DEST_airportID,DEST_ICAO,DEST_Timezone,DEST_TZ,CRS_DEP_DATETIME,ACT_DEP_DATETIME,EARLIER_DATETIME,CRS_ARR_DATETIME,ACT_ARR_DATETIME,ORIGIN_near_station
2015,1,2,1,7,2015-02-01,AA,19805,AA,N016AA,349,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,2105,2116,11.0,11.0,0.0,0,2100-2159,12.0,2128,2252,9.0,2300,2301,1.0,1.0,0.0,0,2300-2359,0.0,null,0.0,115.0,105.0,84.0,1.0,594.0,3,null,null,null,null,null,247,3682,KATL,-5,America/New_York,3576,KMIA,-5,America/New_York,2015-02-02T02:05:00.000+0000,2015-02-02T02:16:00.000+0000,2015-02-02T00:05:00.000+0000,2015-02-02T04:00:00.000+0000,2015-02-02T04:01:00.000+0000,72219013874
2015,1,2,2,1,2015-02-02,AA,19805,AA,N019AA,349,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,2105,2056,-9.0,0.0,0.0,-1,2100-2159,24.0,2120,2242,6.0,2300,2248,-12.0,0.0,0.0,-1,2300-2359,0.0,null,0.0,115.0,112.0,82.0,1.0,594.0,3,null,null,null,null,null,248,3682,KATL,-5,America/New_York,3576,KMIA,-5,America/New_York,2015-02-03T02:05:00.000+0000,2015-02-03T01:56:00.000+0000,2015-02-03T00:05:00.000+0000,2015-02-03T04:00:00.000+0000,2015-02-03T03:48:00.000+0000,72219013874
2015,1,2,3,2,2015-02-03,AA,19805,AA,N008AA,349,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,2105,2058,-7.0,0.0,0.0,-1,2100-2159,19.0,2117,2243,7.0,2300,2250,-10.0,0.0,0.0,-1,2300-2359,0.0,null,0.0,115.0,112.0,86.0,1.0,594.0,3,null,null,null,null,null,249,3682,KATL,-5,America/New_York,3576,KMIA,-5,America/New_York,2015-02-04T02:05:00.000+0000,2015-02-04T01:58:00.000+0000,2015-02-04T00:05:00.000+0000,2015-02-04T04:00:00.000+0000,2015-02-04T03:50:00.000+0000,72219013874
2015,1,2,4,3,2015-02-04,AA,19805,AA,N024AA,349,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,2105,2057,-8.0,0.0,0.0,-1,2100-2159,23.0,2120,2245,3.0,2300,2248,-12.0,0.0,0.0,-1,2300-2359,0.0,null,0.0,115.0,111.0,85.0,1.0,594.0,3,null,null,null,null,null,250,3682,KATL,-5,America/New_York,3576,KMIA,-5,America/New_York,2015-02-05T02:05:00.000+0000,2015-02-05T01:57:00.000+0000,2015-02-05T00:05:00.000+0000,2015-02-05T04:00:00.000+0000,2015-02-05T03:48:00.000+0000,72219013874
2015,1,2,6,5,2015-02-06,AA,19805,AA,N006AA,349,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,2105,2057,-8.0,0.0,0.0,-1,2100-2159,25.0,2122,2242,5.0,2300,2247,-13.0,0.0,0.0,-1,2300-2359,0.0,null,0.0,115.0,110.0,80.0,1.0,594.0,3,null,null,null,null,null,251,3682,KATL,-5,America/New_York,3576,KMIA,-5,America/New_York,2015-02-07T02:05:00.000+0000,2015-02-07T01:57:00.000+0000,2015-02-07T00:05:00.000+0000,2015-02-07T04:00:00.000+0000,2015-02-07T03:47:00.000+0000,72219013874
2015,1,2,7,6,2015-02-07,AA,19805,AA,N012AA,349,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13303,1330303,32467,MIA,"Miami, FL",FL,12,Florida,33,2105,2058,-7.0,0.0,0.0,-1,2100-2159,12.0,2110,2240,8.0,2300,2248,-12.0,0.0,0.0,-1,2300-2359,0.0,null,0.0,115.0,110.0,90.0,1.0,594.0,3,null,null,null,null

In [0]:
print(df_airlines_airport.count(), df_airlines_airport_station_processed.count())

31746841 31746841

In [0]:
# attach the nearest weather station to destination airport
df_airlines_airport_station_processed = df_airlines_airport_station_processed \
                                       .join(df_station_processed, 
                                             df_airlines_airport_station_processed.DEST_ICAO == df_station_processed.neighbor_call, 'left') \
                                       .select(df_airlines_airport_station_processed['*'], \
                                               df_station_processed['station_id'].alias('DEST_near_station'))
#display(df_airlines_airport_station_processed.limit(10))

In [0]:
print(df_weather_station.count())

9368329

### Find the latest weather timestamp before 2hrs of leadtime for each flight

In [0]:
nearest_weather_time_ORIGIN = df_airlines_airport_station_processed \
                      .join(df_weather_select, df_airlines_airport_station_processed.ORIGIN_near_station == df_weather.STATION, how='left') \
                      .where(df_airlines_airport.EARLIER_DATETIME >= df_weather_select.DATE) \
                      .groupBy(df_airlines_airport['EARLIER_DATETIME'], df_airlines_airport_station_processed['flightID'], \
                              df_airlines_airport_station_processed['ORIGIN_near_station']) \
                      .agg(f.max(df_weather_select.DATE).alias('ORIGIN_weather_time'))

In [0]:
display(nearest_weather_time_ORIGIN.limit(10))

In [0]:
print(nearest_weather_time_ORIGIN.count())

31607585

In [0]:
# Join df_weather with df_airlines_airpor_station_processed for origin airport based on 'flightID', 'nearest_weather_time_ORIGIN'
df_intermediate = df_airlines_airport_station_processed \
                 .join(nearest_weather_time_ORIGIN, \
                       df_airlines_airport_station_processed.flightID == nearest_weather_time_ORIGIN.flightID, how='left') \
                 .join(df_weather_select, 
                       (df_airlines_airport_station_processed.ORIGIN_near_station == df_weather_select.STATION) & \
                       (nearest_weather_time_ORIGIN.ORIGIN_weather_time == df_weather_select.DATE) & \
                       (nearest_weather_time_ORIGIN.ORIGIN_near_station == df_weather_select.STATION), how='left') \
                 .select(df_airlines_airport_station_processed['*'],df_weather_select['*']) \
                 .withColumnRenamed('neighbor_id', 'ORIG_neighbor_id') \
                 .withColumnRenamed('neighbor_name', 'ORIG_neighbor_name') \
                 .withColumnRenamed('neighbor_state', 'ORIG_neighbor_state') \
                 .withColumnRenamed('neighbor_call', 'ORIG_neighbor_call') \
                 .withColumnRenamed('distance_to_neighbor', 'ORIG_distance_to_neighbor') \
                 .withColumnRenamed('DATE', 'ORIG_weather_date') \
                 .withColumnRenamed('LATITUDE', 'ORIG_LATITUDE') \
                 .withColumnRenamed('LONGITUDE', 'ORIG_LONGITUDE') \
                 .withColumnRenamed('ELEVATION', 'ORIG_ELEVATION') \
                 .withColumnRenamed('direction_angle', 'ORIG_direction_angle') \
                 .withColumnRenamed('speed', 'ORIG_speed') \
                 .withColumnRenamed('ceiling_height', 'ORIG_ceiling_height') \
                 .withColumnRenamed('vis_distance', 'ORIG_vis_distance') \
                 .withColumnRenamed('variability', 'ORIG_variability') \
                 .withColumnRenamed('air_temp', 'ORIG_air_temp') \
                 .withColumnRenamed('dew_point', 'ORIG_dew_point') \
                 .withColumnRenamed('sea_level_pressure', 'ORIG_sea_level_pressure') \
                 .withColumnRenamed('precipitation_hrs', 'ORIG_precipitation_hrs') \
                 .withColumnRenamed('precipitation_depth', 'ORIG_precipitation_depth') \
                 .drop('neighbor_lat', 'neighbor_lon', 'STATION', 'NAME', \
                       'station_id', 'lat','lon','direction_quality','type','speed_quality','ceiling_determination','CAVOK','distance_quality', 'ceiling_quality', \
                       'quality_variability','air_temp_quality', 'dew_point_quality','sea_level_pressure_quality','precipitation_quality')
                       
#display(df_intermediate.limit(50))

In [0]:
nearest_weather_time_DEST = df_airlines_airport_station_processed \
                      .join(df_weather_select, df_airlines_airport_station_processed.DEST_near_station == df_weather.STATION, how='left') \
                      .where(df_airlines_airport.EARLIER_DATETIME >= df_weather_select.DATE) \
                      .groupBy(df_airlines_airport['EARLIER_DATETIME'], df_airlines_airport_station_processed['flightID'],
                              df_airlines_airport_station_processed['DEST_near_station']) \
                      .agg(f.max(df_weather_select.DATE).alias('DEST_weather_time'))

In [0]:
# Join df_weather with df_airlines_airpor_station_processed for destination airport based on 'flightID', 'nearest_weather_time_DEST'
df_all_joined = df_intermediate \
                 .join(nearest_weather_time_DEST, \
                       df_intermediate.flightID == nearest_weather_time_DEST.flightID, how='left') \
                 .join(df_weather_select, 
                       (df_intermediate.DEST_near_station == df_weather_select.STATION) & \
                       (nearest_weather_time_DEST.DEST_weather_time == df_weather_select.DATE) & \
                       (nearest_weather_time_DEST.DEST_near_station == df_weather_select.STATION), how='left') \
                 .select(df_intermediate['*'],df_weather_select['*']) \
                 .withColumnRenamed('neighbor_id', 'DEST_neighbor_id') \
                 .withColumnRenamed('neighbor_name', 'DEST_neighbor_name') \
                 .withColumnRenamed('neighbor_state', 'DEST_neighbor_state') \
                 .withColumnRenamed('neighbor_call', 'DEST_neighbor_call') \
                 .withColumnRenamed('distance_to_neighbor', 'DEST_distance_to_neighbor') \
                 .withColumnRenamed('DATE', 'DEST_weather_date') \
                 .withColumnRenamed('LATITUDE', 'DEST_LATITUDE') \
                 .withColumnRenamed('LONGITUDE', 'DEST_LONGITUDE') \
                 .withColumnRenamed('ELEVATION', 'DEST_ELEVATION') \
                 .withColumnRenamed('direction_angle', 'DEST_direction_angle') \
                 .withColumnRenamed('speed', 'DEST_speed') \
                 .withColumnRenamed('ceiling_height', 'DEST_ceiling_height') \
                 .withColumnRenamed('vis_distance', 'DEST_vis_distance') \
                 .withColumnRenamed('variability', 'DEST_variability') \
                 .withColumnRenamed('air_temp', 'DEST_air_temp') \
                 .withColumnRenamed('dew_point', 'DEST_dew_point') \
                 .withColumnRenamed('sea_level_pressure', 'DEST_sea_level_pressure') \
                 .withColumnRenamed('precipitation_hrs', 'DEST_precipitation_hrs') \
                 .withColumnRenamed('precipitation_depth', 'DEST_precipitation_depth') \
                 .drop('neighbor_lat', 'neighbor_lon', 'STATION', 'NAME', \
                       'station_id', 'lat','lon','direction_quality','type','speed_quality','ceiling_determination','CAVOK','distance_quality', 'ceiling_quality', \
                       'quality_variability','air_temp_quality', 'dew_point_quality','sea_level_pressure_quality','precipitation_quality')
                       

#Save data to Cloud

In [0]:
spark.conf.set("spark.databricks.queryWatchdog.outputRatioThreshold", 30000)

In [0]:
# save combined dataset
if timeframe == "5y":
    df_all_joined.write.mode('overwrite').parquet(f"/mnt/team11/all_flight_weather_5y_v2/")
elif timeframe == "3m":
    df_all_joined.write.mode('overwrite').parquet(f"/mnt/team11/all_flight_weather_3m/")
else:
    df_all_joined.write.mode('overwrite').parquet(f"/mnt/team11/all_flight_weather_5y/" + timeframe + "parquet")

In [0]:
display(dbutils.fs.ls("/mnt/team11/all_flight_weather_5y_v2/"))

path,name,size
dbfs:/mnt/team11/all_flight_weather_5y_v2/_SUCCESS,_SUCCESS,0
dbfs:/mnt/team11/all_flight_weather_5y_v2/_committed_3573657853771596369,_committed_3573657853771596369,20224
dbfs:/mnt/team11/all_flight_weather_5y_v2/_started_3573657853771596369,_started_3573657853771596369,0
dbfs:/mnt/team11/all_flight_weather_5y_v2/part-00000-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2539-1-c000.snappy.parquet,part-00000-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2539-1-c000.snappy.parquet,15738672
dbfs:/mnt/team11/all_flight_weather_5y_v2/part-00001-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2654-1-c000.snappy.parquet,part-00001-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2654-1-c000.snappy.parquet,15380075
dbfs:/mnt/team11/all_flight_weather_5y_v2/part-00002-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2648-1-c000.snappy.parquet,part-00002-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2648-1-c000.snappy.parquet,15366760
dbfs:/mnt/team11/all_flight_weather_5y_v2/part-00003-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2531-1-c000.snappy.parquet,part-00003-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2531-1-c000.snappy.parquet,15721160
dbfs:/mnt/team11/all_flight_weather_5y_v2/part-00004-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2699-1-c000.snappy.parquet,part-00004-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2699-1-c000.snappy.parquet,15183418
dbfs:/mnt/team11/all_flight_weather_5y_v2/part-00005-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2516-1-c000.snappy.parquet,part-00005-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2516-1-c000.snappy.parquet,15824948
dbfs:/mnt/team11/all_flight_weather_5y_v2/part-00006-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2671-1-c000.snappy.parquet,part-00006-tid-3573657853771596369-073fdaa8-2e78-4b89-a8d0-baf4d3dc9950-2671-1-c000.snappy.parquet,15410488


### Data reconciliation

Raw Data:

2015: the number of records in airline data is  5819079 and the number of attributes is  62

2016: the number of records in airline data is  5617658 and the number of attributes is  62

2017: the number of records in airline data is  5674621 and the number of attributes is  62

2018: the number of records in airline data is  7213446 and the number of attributes is  62

2019: the number of records in airline data is  7422037 and the number of attributes is  62


Removed diverted and cancelled flights:

2015: the number of records in airline data is  5714008

2016: the number of records in airline data is  5538145

2017: the number of records in airline data is  5579411

2018: the number of records in airline data is  7079005

2019: the number of records in airline data is  7268232

Weather and station data mapped:

2015: 5714008

2016: 5538145

2017: 5579411

2018: 7079005

2019: 7268232

Diverted and cancelled flights are just about 2% of total flights. No data removed otherwise. The total number of rows are 31178801.